In [22]:
import numpy as np
import os
import pandas as pd

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [23]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'         )
dirIntermediate = os.path.join(dirWork, r'intermediate'  )
dirResults      = os.path.join(dirWork, r'results'       )
dirParams       = os.path.join(dirWork, r'params'        )
dirWebApp       = os.path.join(dirResults, r'webapp_data')

In [24]:
dfTruckData = pd.read_csv(os.path.join(dirIntermediate, r'TruckData_Combined_2012to2023.csv'))
dfSegFac  = pd.read_csv(os.path.join(dirIntermediate, 'SegmentsMatchedToFactorGroups.csv'))
sdfSegments = pd.DataFrame.spatial.from_featureclass(os.path.join(dirWork,  r"arcgis\UDOT-Segments.gdb\Master_Segs_20210811"))

In [25]:
#read in year groups to be able to produce truck factors by different year ranges
prmYearGroups = pd.read_csv(os.path.join(dirParams, r'year_groups.csv'))
prmYearToYearGroups = pd.read_csv(os.path.join(dirParams, r'year_to_year_groups.csv'))

In [26]:
dfSegments = sdfSegments[['SEGID','LRS_ROUTE','BMP','EMP']].copy()

dfSegments['ROUTE']=dfSegments['LRS_ROUTE'].str[0:4]
dfSegments['MP_MID'] = dfSegments['BMP']/2 + dfSegments['EMP']/2

dfSegments = dfSegments.drop(columns=(['LRS_ROUTE']))

#filter by last 5 years
#dfTruckData = dfTruckData[((dfTruckData['YEAR']>=2015) & (dfTruckData['YEAR']<=2019))]

dfTruckDataWithYG = pd.DataFrame.merge(dfTruckData,prmYearToYearGroups,on='YEAR')

dfTruckDataWithYG = dfTruckDataWithYG.drop(columns=(['DESC','SegID']))

dfSegmentsWithTruckData = pd.DataFrame.merge(dfSegments,dfTruckDataWithYG,on='ROUTE')

#filter by segments with midpoint milepost with data range begmp to endmp
dfSegmentsWithTruckData = dfSegmentsWithTruckData[((dfSegmentsWithTruckData['MP_MID']>=dfSegmentsWithTruckData['BEGMP']) & (dfSegmentsWithTruckData['MP_MID']<dfSegmentsWithTruckData['ENDMP']))]

dfSegmentsWithTruckData['VMT']       = dfSegmentsWithTruckData['AADT']  * (dfSegmentsWithTruckData['EMP'] - dfSegmentsWithTruckData['BMP'])
dfSegmentsWithTruckData['SUTRKxVMT'] = dfSegmentsWithTruckData['SUTRK'] * dfSegmentsWithTruckData['VMT']
dfSegmentsWithTruckData['CUTRKxVMT'] = dfSegmentsWithTruckData['CUTRK'] * dfSegmentsWithTruckData['VMT']

dfSegmentsWithTruckDataAggregated = dfSegmentsWithTruckData.groupby(['SEGID','YEARGROUP'],as_index=False).agg(sumVMT=('VMT','sum'),sumSUTRKxVMT=('SUTRKxVMT','sum'),sumCUTRKxVMT=('CUTRKxVMT','sum'))
dfSegmentsWithTruckDataAggregated['SUTRK_wght'] = dfSegmentsWithTruckDataAggregated['sumSUTRKxVMT'] /  dfSegmentsWithTruckDataAggregated['sumVMT']
dfSegmentsWithTruckDataAggregated['CUTRK_wght'] = dfSegmentsWithTruckDataAggregated['sumCUTRKxVMT'] /  dfSegmentsWithTruckDataAggregated['sumVMT']
dfSegmentsWithTruckDataAggregated = dfSegmentsWithTruckDataAggregated[dfSegmentsWithTruckDataAggregated['SUTRK_wght']>0]

dfSegmentsTruck = dfSegmentsWithTruckDataAggregated[['SEGID','YEARGROUP','SUTRK_wght','CUTRK_wght']]
dfSegmentsTruck.columns = ('SEGID','YEARGROUP','SUTRKseg','CUTRKseg')

display(dfSegments)
display(dfTruckDataWithYG)
display(dfSegmentsWithTruckData)
display(dfSegmentsWithTruckDataAggregated)
display(dfSegmentsTruck)


,SEGID,BMP,EMP,ROUTE,MP_MID
0,0163_000.0,0.0,0.458,0163,0.229
1,3225_000.0,0.0,1.68,3225,0.84
2,3142_000.0,0.0,0.291,3142,0.1455
3,3149_000.0,0.031,0.289,3149,0.16
4,0007_006.1,6.125,7.308,0007,6.7165
...,...,...,...,...,...
8552,2080_009.3,9.272,9.636,2080,9.454
8553,2161_003.2,3.207,3.772,2161,3.4895
8554,WFRC_8220,0.0,0.0,0,0.0
8555,2161_003.4,3.4,3.772,2161,3.586


,YEAR,ROUTE,BEGMP,ENDMP,AADT,SUTRK,CUTRK,YEARGROUP
0,2013,0006,0.000,46.0100,330.0,0.250000,0.230000,2013
1,2013,0006,0.000,46.0100,330.0,0.250000,0.230000,2013-2017
2,2013,0006,0.000,46.0100,330.0,0.250000,0.230000,All Years
3,2013,0006,46.010,77.5400,340.0,0.180000,0.330000,2013
4,2013,0006,46.010,77.5400,340.0,0.180000,0.330000,2013-2017
...,...,...,...,...,...,...,...,...
69269,2023,0491,2.031,17.0694,3558.0,0.063942,0.330758,2019-2023
69270,2023,0491,2.031,17.0694,3558.0,0.063942,0.330758,All Years
69271,2023,0089,0.000,2.9700,5308.0,0.115070,0.057076,2023
69272,2023,0089,0.000,2.9700,5308.0,0.115070,0.057076,2019-2023


,SEGID,BMP,EMP,ROUTE,MP_MID,YEAR,BEGMP,ENDMP,AADT,SUTRK,CUTRK,YEARGROUP,VMT,SUTRKxVMT,CUTRKxVMT
0,0163_000.0,0.0,0.458,0163,0.229,2013,0.000,19.2500,2080.0,0.250000,0.020000,2013,952.64,238.16,19.0528
1,0163_000.0,0.0,0.458,0163,0.229,2013,0.000,19.2500,2080.0,0.250000,0.020000,2013-2017,952.64,238.16,19.0528
2,0163_000.0,0.0,0.458,0163,0.229,2013,0.000,19.2500,2080.0,0.250000,0.020000,All Years,952.64,238.16,19.0528
12,0163_000.0,0.0,0.458,0163,0.229,2014,0.000,19.2500,2110.0,0.240000,0.020000,2014,966.38,231.9312,19.3276
13,0163_000.0,0.0,0.458,0163,0.229,2014,0.000,19.2500,2110.0,0.240000,0.020000,2013-2017,966.38,231.9312,19.3276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3875779,0135_000.4,0.364,0.726,0135,0.545,2021,0.396,0.7310,32806.0,0.110711,0.124063,2019-2023,11875.772,1314.783997,1473.339995
3875780,0135_000.4,0.364,0.726,0135,0.545,2021,0.396,0.7310,32806.0,0.110711,0.124063,All Years,11875.772,1314.783997,1473.339995
3875784,0135_000.4,0.364,0.726,0135,0.545,2023,0.396,0.7312,33598.0,0.110716,0.124068,2023,12162.476,1346.576203,1508.971599
3875785,0135_000.4,0.364,0.726,0135,0.545,2023,0.396,0.7312,33598.0,0.110716,0.124068,2019-2023,12162.476,1346.576203,1508.971599


,SEGID,YEARGROUP,sumVMT,sumSUTRKxVMT,sumCUTRKxVMT,SUTRK_wght,CUTRK_wght
0,0006_000.0,2013,219.45,54.8625,50.4735,0.25,0.23
1,0006_000.0,2013-2017,1239.56,307.456422,285.764972,0.248037,0.230537
2,0006_000.0,2014,232.75,55.86,53.5325,0.24,0.23
3,0006_000.0,2015,249.375,62.34375,57.35625,0.25,0.23
4,0006_000.0,2015-2019,1337.315,334.00818,309.595626,0.24976,0.231505
...,...,...,...,...,...,...,...
42670,089A_000.0,2016,13519.4,811.164,675.97,0.06,0.05
42671,089A_000.0,2017,14636.22,818.880758,660.765621,0.055949,0.045146
42673,089A_000.0,2019,15288.678,2124.122831,920.067872,0.138934,0.06018
42674,089A_000.0,2019-2023,15288.678,2124.122831,920.067872,0.138934,0.06018


,SEGID,YEARGROUP,SUTRKseg,CUTRKseg
0,0006_000.0,2013,0.25,0.23
1,0006_000.0,2013-2017,0.248037,0.230537
2,0006_000.0,2014,0.24,0.23
3,0006_000.0,2015,0.25,0.23
4,0006_000.0,2015-2019,0.24976,0.231505
...,...,...,...,...
42670,089A_000.0,2016,0.06,0.05
42671,089A_000.0,2017,0.055949,0.045146
42673,089A_000.0,2019,0.138934,0.06018
42674,089A_000.0,2019-2023,0.138934,0.06018


In [27]:
#group segments data into Station Groups so that they can be applied to segments that don't have truck data (typically Federal-Aid Routes)

dfSegTruckGroup = pd.DataFrame.merge(dfSegmentsWithTruckData,dfSegFac,on='SEGID')

dfSegTruckGroup = dfSegTruckGroup.drop(columns=(['ROUTE','MP_MID','BEGMP','ENDMP']))

dfSegTruckGroup['VMT2019']   = dfSegTruckGroup['AADT2019'] * (dfSegTruckGroup['EMP'] - dfSegTruckGroup['BMP'])
dfSegTruckGroup['SUTRKxVMT'] = dfSegTruckGroup['SUTRK'] * dfSegTruckGroup['VMT2019']
dfSegTruckGroup['CUTRKxVMT'] = dfSegTruckGroup['CUTRK'] * dfSegTruckGroup['VMT2019']

#aggregate by groups to get values to segments without truck percentage values (typically federal aid and non-state routes)
dfSegTruckGroupAggregated = dfSegTruckGroup.groupby(['STATIONGROUP','YEARGROUP'],as_index=False).agg(sumVMT2019=('VMT2019','sum'),sumSUTRKxVMT=('SUTRKxVMT','sum'),sumCUTRKxVMT=('CUTRKxVMT','sum'))
dfSegTruckGroupAggregated['SUTRK_wght'] = dfSegTruckGroupAggregated['sumSUTRKxVMT'] /  dfSegTruckGroupAggregated['sumVMT2019']
dfSegTruckGroupAggregated['CUTRK_wght'] = dfSegTruckGroupAggregated['sumCUTRKxVMT'] /  dfSegTruckGroupAggregated['sumVMT2019']

dfGroupTruck = dfSegTruckGroupAggregated[['STATIONGROUP','YEARGROUP','SUTRK_wght','CUTRK_wght']]
dfGroupTruck.columns = ('STATIONGROUP','YEARGROUP','SUTRKgrp','CUTRKgrp')

display(dfGroupTruck)

#pd.set_option('display.max_rows', dfGroupTruck.shape[0]+1)
#display(dfGroupTruck)
#pd.set_option('display.max_rows', 10)


,STATIONGROUP,YEARGROUP,SUTRKgrp,CUTRKgrp
0,CO0,2013,0.065502,0.043508
1,CO0,2013-2017,0.063965,0.048449
2,CO0,2014,0.058023,0.045629
3,CO0,2015,0.067624,0.050211
4,CO0,2015-2019,0.078795,0.051343
...,...,...,...,...
1325,XX4,2019-2023,0.15426,0.038498
1326,XX4,2020,0.185919,0.04698
1327,XX4,2021,0.141955,0.035625
1328,XX4,2023,0.150112,0.033866


In [28]:
dfGroupTruck['STATIONGROUP'].unique().size

95

In [29]:
dfGroupTruck.groupby(['YEARGROUP'],as_index=False).agg(sgcount=('STATIONGROUP','size'))

,YEARGROUP,sgcount
0,2013,95
1,2013-2017,95
2,2014,95
3,2015,95
4,2015-2019,95
5,2016,95
6,2017,95
7,2018,95
8,2019,95
9,2019-2023,95


In [30]:
dfSegmentsTruck

,SEGID,YEARGROUP,SUTRKseg,CUTRKseg
0,0006_000.0,2013,0.25,0.23
1,0006_000.0,2013-2017,0.248037,0.230537
2,0006_000.0,2014,0.24,0.23
3,0006_000.0,2015,0.25,0.23
4,0006_000.0,2015-2019,0.24976,0.231505
...,...,...,...,...
42670,089A_000.0,2016,0.06,0.05
42671,089A_000.0,2017,0.055949,0.045146
42673,089A_000.0,2019,0.138934,0.06018
42674,089A_000.0,2019-2023,0.138934,0.06018


In [31]:
#create records of all segments with all possible year groups
dfSegFacYG = pd.DataFrame.merge(dfSegFac,prmYearGroups,how='cross')

#add truck data where it already exists for segments
dfSegFacWithTrkSeg = pd.DataFrame.merge(dfSegFacYG,dfSegmentsTruck,on=('SEGID','YEARGROUP'),how='left')

#add truck data where it exists for STATIONGROUP
dfSegFacWithTrkSegTrkGrp = pd.DataFrame.merge(dfSegFacWithTrkSeg,dfGroupTruck,on=('STATIONGROUP','YEARGROUP'),how='left')

#for those without a match, fill with zeros
dfSegFacWithTrkSegTrkGrp = dfSegFacWithTrkSegTrkGrp.fillna(0)

#display
display(dfSegFacWithTrkSegTrkGrp)

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,YEARGROUP,SUTRKseg,CUTRKseg,SUTRKgrp,CUTRKgrp
0,0006_000.0,415,1,Arterial,Statewide,WD0,2013,0.25,0.23,0.148666,0.149748
1,0006_000.0,415,1,Arterial,Statewide,WD0,2014,0.24,0.23,0.157038,0.149664
2,0006_000.0,415,1,Arterial,Statewide,WD0,2015,0.25,0.23,0.162078,0.153207
3,0006_000.0,415,1,Arterial,Statewide,WD0,2016,0.25,0.23,0.161313,0.153038
4,0006_000.0,415,1,Arterial,Statewide,WD0,2017,0.24961,0.232449,0.142968,0.152093
...,...,...,...,...,...,...,...,...,...,...,...
385465,1760_000.0,713,3,Arterial,Statewide,COT,2023,0.0,0.0,0.10749,0.056124
385466,1760_000.0,713,3,Arterial,Statewide,COT,2013-2017,0.0,0.0,0.124625,0.067325
385467,1760_000.0,713,3,Arterial,Statewide,COT,2015-2019,0.0,0.0,0.122057,0.065614
385468,1760_000.0,713,3,Arterial,Statewide,COT,2019-2023,0.0,0.0,0.111855,0.056501


In [32]:
dfSegFacWithTrk = dfSegFacWithTrkSegTrkGrp.copy()

dfSegFacWithTrk['SUTRUCKS']=dfSegFacWithTrk['SUTRKseg']
dfSegFacWithTrk['CUTRUCKS']=dfSegFacWithTrk['CUTRKseg']

#if no segment truck, use group truck data
dfSegFacWithTrk.loc[(dfSegFacWithTrk['SUTRKseg']==0),'SUTRUCKS'] = dfSegFacWithTrk['SUTRKgrp']
dfSegFacWithTrk.loc[(dfSegFacWithTrk['CUTRKseg']==0),'CUTRUCKS'] = dfSegFacWithTrk['CUTRKgrp']
display(dfSegFacWithTrk)

dfSegFacWithTrk = dfSegFacWithTrk.drop(columns=(['SUTRKseg','SUTRKgrp','CUTRKseg','CUTRKgrp']))

dfSegFacWithTrk['SUTRUCKS'] = round(dfSegFacWithTrk['SUTRUCKS'],4)
dfSegFacWithTrk['CUTRUCKS'] = round(dfSegFacWithTrk['CUTRUCKS'],4)

dfSegFacWithTrk['AADT2019'] = dfSegFacWithTrk['AADT2019'].astype(int)
dfSegFacWithTrk['AREATYPE'] = dfSegFacWithTrk['AREATYPE'].astype(int)
dfSegFacWithTrk

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,YEARGROUP,SUTRKseg,CUTRKseg,SUTRKgrp,CUTRKgrp,SUTRUCKS,CUTRUCKS
0,0006_000.0,415,1,Arterial,Statewide,WD0,2013,0.25,0.23,0.148666,0.149748,0.25,0.23
1,0006_000.0,415,1,Arterial,Statewide,WD0,2014,0.24,0.23,0.157038,0.149664,0.24,0.23
2,0006_000.0,415,1,Arterial,Statewide,WD0,2015,0.25,0.23,0.162078,0.153207,0.25,0.23
3,0006_000.0,415,1,Arterial,Statewide,WD0,2016,0.25,0.23,0.161313,0.153038,0.25,0.23
4,0006_000.0,415,1,Arterial,Statewide,WD0,2017,0.24961,0.232449,0.142968,0.152093,0.24961,0.232449
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385465,1760_000.0,713,3,Arterial,Statewide,COT,2023,0.0,0.0,0.10749,0.056124,0.10749,0.056124
385466,1760_000.0,713,3,Arterial,Statewide,COT,2013-2017,0.0,0.0,0.124625,0.067325,0.124625,0.067325
385467,1760_000.0,713,3,Arterial,Statewide,COT,2015-2019,0.0,0.0,0.122057,0.065614,0.122057,0.065614
385468,1760_000.0,713,3,Arterial,Statewide,COT,2019-2023,0.0,0.0,0.111855,0.056501,0.111855,0.056501


,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,YEARGROUP,SUTRUCKS,CUTRUCKS
0,0006_000.0,415,1,Arterial,Statewide,WD0,2013,0.25,0.23
1,0006_000.0,415,1,Arterial,Statewide,WD0,2014,0.24,0.23
2,0006_000.0,415,1,Arterial,Statewide,WD0,2015,0.25,0.23
3,0006_000.0,415,1,Arterial,Statewide,WD0,2016,0.25,0.23
4,0006_000.0,415,1,Arterial,Statewide,WD0,2017,0.2496,0.2324
...,...,...,...,...,...,...,...,...,...
385465,1760_000.0,713,3,Arterial,Statewide,COT,2023,0.1075,0.0561
385466,1760_000.0,713,3,Arterial,Statewide,COT,2013-2017,0.1246,0.0673
385467,1760_000.0,713,3,Arterial,Statewide,COT,2015-2019,0.1221,0.0656
385468,1760_000.0,713,3,Arterial,Statewide,COT,2019-2023,0.1119,0.0565


In [33]:
#dfSegFacWithTrk.to_csv(os.path.join(dirIntermediate, r'SegmentTruckFactors.csv'),index=False)
#dfSegFacWithTrk

# after further investigation seen below. going to use 2019 truck data for truck factors
dfSegFacWithTrk_export = dfSegFacWithTrk[dfSegFacWithTrk['YEARGROUP']=='2019']
dfSegFacWithTrk_export = dfSegFacWithTrk_export.drop(columns=('YEARGROUP'))

dfSegFacWithTrk_export.to_csv(os.path.join(dirIntermediate, r'SegmentTruckFactors.csv'),index=False)

display(dfSegFacWithTrk_export)
display(dfSegFacWithTrk_export[dfSegFacWithTrk_export['SEGID']=='WFRC_8223'])

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,SUTRUCKS,CUTRUCKS
6,0006_000.0,415,1,Arterial,Statewide,WD0,0.2496,0.2324
21,0006_000.0,415,1,Arterial,Statewide,SNP,0.2496,0.2324
36,0006_000.0,415,1,Arterial,Statewide,COP,0.2496,0.2324
51,0006_000.7,415,1,Arterial,Statewide,WD0,0.2496,0.2324
66,0006_000.7,415,1,Arterial,Statewide,SNP,0.2496,0.2324
...,...,...,...,...,...,...,...,...
385401,0056_060.4,13242,3,Arterial,Statewide,SNT,0.1143,0.0577
385416,0056_060.4,13242,3,Arterial,Statewide,COT,0.1143,0.0577
385431,1760_000.0,713,3,Arterial,Statewide,WD2,0.1124,0.064
385446,1760_000.0,713,3,Arterial,Statewide,SNT,0.1143,0.0577


,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,SUTRUCKS,CUTRUCKS


In [34]:
dfSegFacWithTrk_export[dfSegFacWithTrk_export['SEGID']=='0175_000.0']

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,SUTRUCKS,CUTRUCKS
111426,0175_000.0,32668,3,Arterial,Statewide,WD2,0.2517,0.037
111441,0175_000.0,32668,3,Arterial,Statewide,SNT,0.2517,0.037
111456,0175_000.0,32668,3,Arterial,Statewide,COT,0.2517,0.037


# Investigating Segments that have SU/CU trucks switched in earlier years and fixed in 2019

Segment 0175_000.0 and others on this route appear to have error in pre-2019 data where SUTRUCKS and CUTRUCKS are switched
at least that's what it looks like when considering 2019 and expections.
This route is 11400 South west of State Street

In [35]:
df = dfSegFacWithTrk[dfSegFacWithTrk['SEGID']=='0175_000.0']
df

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,YEARGROUP,SUTRUCKS,CUTRUCKS
111420,0175_000.0,32668,3,Arterial,Statewide,WD2,2013,0.09,0.28
111421,0175_000.0,32668,3,Arterial,Statewide,WD2,2014,0.08,0.28
111422,0175_000.0,32668,3,Arterial,Statewide,WD2,2015,0.09,0.29
111423,0175_000.0,32668,3,Arterial,Statewide,WD2,2016,0.09,0.29
111424,0175_000.0,32668,3,Arterial,Statewide,WD2,2017,0.0858,0.2865
111425,0175_000.0,32668,3,Arterial,Statewide,WD2,2018,0.0858,0.2865
111426,0175_000.0,32668,3,Arterial,Statewide,WD2,2019,0.2517,0.037
111427,0175_000.0,32668,3,Arterial,Statewide,WD2,2020,0.2517,0.037
111428,0175_000.0,32668,3,Arterial,Statewide,WD2,2021,0.2517,0.037
111429,0175_000.0,32668,3,Arterial,Statewide,WD2,2022,0.0,0.0


In [36]:
prmYearGroups['YEARGROUP'].tolist()

['2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '2013-2017',
 '2015-2019',
 '2019-2023',
 'All Years']

In [37]:
#show only CO groups
dfSU = dfSegFacWithTrk[dfSegFacWithTrk['STATIONGROUP'].str[0:2]=='CO']
dfSU_pivot = dfSU.pivot(index='SEGID', columns='YEARGROUP', values='SUTRUCKS')
dfSU_pivot = dfSU_pivot[prmYearGroups['YEARGROUP'].tolist()]
dfSU_pivot['COMPARE'] = dfSU_pivot['2019'] - dfSU_pivot['2015-2019']
display(dfSU_pivot)

dfCU = dfSegFacWithTrk[dfSegFacWithTrk['STATIONGROUP'].str[0:2]=='CO']
dfCU_pivot = dfCU.pivot(index='SEGID', columns='YEARGROUP', values='CUTRUCKS')
dfCU_pivot = dfCU_pivot[prmYearGroups['YEARGROUP'].tolist()]
dfCU_pivot['COMPARE'] = dfCU_pivot['2019'] - dfCU_pivot['2015-2019']
display(dfCU_pivot)

YEARGROUP,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2013-2017,2015-2019,2019-2023,All Years,COMPARE
SEGID,,,,,,,,,,,,,,,,
0006_000.0,0.25,0.24,0.25,0.25,0.2496,0.2496,0.2496,0.2496,0.2489,0.0,0.2496,0.248,0.2498,0.2494,0.2488,-0.0002
0006_000.7,0.25,0.24,0.25,0.25,0.2496,0.2496,0.2496,0.2496,0.2489,0.0,0.2496,0.248,0.2498,0.2494,0.2488,-0.0002
0006_016.0,0.25,0.24,0.25,0.25,0.2496,0.2496,0.2496,0.2496,0.2489,0.0,0.2496,0.248,0.2498,0.2494,0.2488,-0.0002
0006_046.0,0.18,0.17,0.17,0.17,0.1751,0.1751,0.1751,0.1751,0.1745,0.0,0.1751,0.1728,0.1729,0.1749,0.1739,0.0022
0006_060.2,0.18,0.17,0.17,0.17,0.1751,0.1751,0.1751,0.1751,0.1745,0.0,0.1751,0.1728,0.1729,0.1749,0.1739,0.0022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WFRC_8214,0.1819,0.1838,0.1882,0.1926,0.1866,0.1865,0.1867,0.1889,0.1915,0.0,0.1875,0.1866,0.1881,0.1886,0.1874,-0.0014
WFRC_8215,0.1282,0.1467,0.1491,0.1428,0.1227,0.1209,0.1278,0.1287,0.1234,0.0,0.1236,0.1379,0.1327,0.1259,0.1314,-0.0049
WFRC_8217,0.1282,0.1467,0.1491,0.1428,0.1227,0.1209,0.1278,0.1287,0.1234,0.0,0.1236,0.1379,0.1327,0.1259,0.1314,-0.0049


YEARGROUP,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2013-2017,2015-2019,2019-2023,All Years,COMPARE
SEGID,,,,,,,,,,,,,,,,
0006_000.0,0.23,0.23,0.23,0.23,0.2324,0.2324,0.2324,0.2324,0.2321,0.0,0.2324,0.2305,0.2315,0.2323,0.2315,0.0009
0006_000.7,0.23,0.23,0.23,0.23,0.2324,0.2324,0.2324,0.2324,0.2321,0.0,0.2324,0.2305,0.2315,0.2323,0.2315,0.0009
0006_016.0,0.23,0.23,0.23,0.23,0.2324,0.2324,0.2324,0.2324,0.2321,0.0,0.2324,0.2305,0.2315,0.2323,0.2315,0.0009
0006_046.0,0.33,0.33,0.33,0.33,0.3338,0.3338,0.3338,0.3338,0.3349,0.0,0.3338,0.3307,0.3322,0.3341,0.3324,0.0016
0006_060.2,0.33,0.33,0.33,0.33,0.3338,0.3338,0.3338,0.3338,0.3349,0.0,0.3338,0.3307,0.3322,0.3341,0.3324,0.0016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WFRC_8214,0.1336,0.1322,0.1366,0.1419,0.141,0.1411,0.1414,0.1424,0.1429,0.0,0.1519,0.1371,0.1404,0.1447,0.1405,0.001
WFRC_8215,0.091,0.0873,0.0915,0.092,0.0909,0.0904,0.0879,0.0881,0.0849,0.0,0.0922,0.0906,0.0905,0.0883,0.0896,-0.0026
WFRC_8217,0.091,0.0873,0.0915,0.092,0.0909,0.0904,0.0879,0.0881,0.0849,0.0,0.0922,0.0906,0.0905,0.0883,0.0896,-0.0026


In [38]:
import pandas as pd

dfSU_pivot_filtered = dfSU_pivot[dfSU_pivot['COMPARE'].abs()>.05]

def condformatvalue(cell_value):

    highlight_blue = 'background-color: lightblue;'
    default = ''

    if type(cell_value) in [float, int]:
        if abs(cell_value) > .3:
            return highlight_blue

        return default

def condformatdiff(cell_value):

    highlight_yellow = 'background-color: yellow;'
    highlight_orange = 'background-color: orange;'
    highlight_red    = 'background-color: red;'
    default = ''

    if type(cell_value) in [float, int]:
        if abs(cell_value) > .15:
            return highlight_red
        elif abs(cell_value) > .10:
            return highlight_orange
        elif abs(cell_value) > .05:
            return highlight_yellow

        return default

#df.style.applymap(condformat, subset=pd.IndexSlice[:, ['2019ComparedToAllYears']])


(dfSU_pivot_filtered
 .style
 .applymap(condformatdiff, subset=pd.IndexSlice[:, ['COMPARE']])
 .applymap(condformatvalue, subset=pd.IndexSlice[:, ['2017','2018']]))


YEARGROUP,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2013-2017,2015-2019,2019-2023,All Years,COMPARE
SEGID,,,,,,,,,,,,,,,,
0006_173.4,0.070000,0.210000,0.210000,0.220000,0.079400,0.076400,0.065700,0.073300,0.073800,0.000000,0.079300,0.158600,0.127000,0.073200,0.112500,-0.061300
0006_173.7,0.070000,0.210000,0.210000,0.220000,0.079400,0.076400,0.065700,0.073300,0.073800,0.000000,0.079300,0.158600,0.127000,0.073200,0.112500,-0.061300
0006_174.0,0.070000,0.210000,0.210000,0.220000,0.079400,0.076400,0.065700,0.073300,0.073800,0.000000,0.079300,0.158400,0.126700,0.073200,0.112400,-0.061000
0006_174.4,0.070000,0.210000,0.210000,0.220000,0.079400,0.076400,0.065700,0.073300,0.073800,0.000000,0.079300,0.158400,0.126700,0.073200,0.112400,-0.061000
0006_177.9,0.250000,0.260000,0.260000,0.260000,0.120600,0.118300,0.101100,0.097400,0.090600,0.000000,0.067600,0.227000,0.168600,0.088400,0.154600,-0.067500
0006_178.8,0.250000,0.260000,0.260000,0.260000,0.120600,0.118300,0.101100,0.097400,0.090600,0.000000,0.067600,0.227000,0.168600,0.088400,0.154600,-0.067500
0006_181.6,0.250000,0.260000,0.260000,0.260000,0.120600,0.118300,0.101100,0.097400,0.090600,0.000000,0.067600,0.227000,0.168600,0.088400,0.154600,-0.067500
0006_184.1,0.250000,0.260000,0.260000,0.260000,0.120600,0.118300,0.101100,0.097400,0.090600,0.000000,0.067600,0.227000,0.168600,0.088400,0.154600,-0.067500
0008_000.0,0.330000,0.320000,0.330000,0.330000,0.142400,0.141800,0.132000,0.129200,0.117800,0.000000,0.086700,0.287800,0.215400,0.115700,0.200900,-0.083400


In [39]:
#create map1 with zoom set to Salt Lake (any city or place name can be used)
map2 = gis.map('Salt Lake')
#map2.basemap='gray'

#join df back to segments

sdfSegmentsWithData = pd.DataFrame.merge(sdfSegments,dfSU_pivot,on='SEGID')

#expression to classify enrollment

arcexp_su = ("var v = $feature.COMPARE;"
             "if      (v<-.12) { return 'neglevel3';}"
             "else if (v<-.08) { return 'neglevel2';}"
             "else if (v<-.04) { return 'neglevel1';}"
             "else if (v< .04) { return '_neutral_';}"
             "else if (v< .08) { return 'poslevel1';}"
             "else if (v< .12) { return 'poslevel2';}"
             "else             { return 'poslevel3';}")

#symbology for enrollment classes
unival_su = [
               {"value":"neglevel3", "label":"Less than -12%", "symbol":{"type":"esriSLS","color":[  0,  0,255,168],"width": 4}},
               {"value":"neglevel2", "label":"-12% to -8%"   , "symbol":{"type":"esriSLS","color":[  0,112,255,168],"width": 3}},
               {"value":"neglevel1", "label":"-8% to -3%"    , "symbol":{"type":"esriSLS","color":[115,178,255,168],"width": 2}},
               {"value":"_neutral_", "label":"-4% to 4%"     , "symbol":{"type":"esriSLS","color":[204,204,204,168],"width": 1}},
               {"value":"poslevel1", "label":"+4% to +8%"    , "symbol":{"type":"esriSLS","color":[255,190,190,168],"width": 2}},
               {"value":"poslevel2", "label":"+8% to +12%"   , "symbol":{"type":"esriSLS","color":[255,127,127,168],"width": 3}},
               {"value":"poslevel3", "label":"More than +12%", "symbol":{"type":"esriSLS","color":[230,  0,  0,168],"width": 4}}
            ]

#define sdf layer
sdfSegmentsWithData.spatial.plot(map_widget = map2,
                                 renderer_type     = 'u-a',
                                 unique_values     = unival_su,
                                 arcade_expression = arcexp_su,
                                 default_symbol    = ""
                                )

map2.layout.height='800px'
map2.legend=True

#add map title
display(Markdown('<h2><center>Segment Single-Unit Changes (2019 vs 2015-2019)</center></h2>'))

#display the map
map2

<h2><center>Segment Single-Unit Changes (2019 vs 2015-2019)</center></h2>

MapView(layout=Layout(height='800px', width='100%'), legend=True)

In [40]:
#create map1 with zoom set to Salt Lake (any city or place name can be used)
map3 = gis.map('Salt Lake')
#map3.basemap='gray'

#join df back to segments

sdfSegmentsWithCUData = pd.DataFrame.merge(sdfSegments,dfCU_pivot,on='SEGID')

#expression to classify enrollment

arcexp_su = ("var v = $feature.COMPARE;"
             "if      (v<-.12) { return 'neglevel3';}"
             "else if (v<-.08) { return 'neglevel2';}"
             "else if (v<-.04) { return 'neglevel1';}"
             "else if (v< .04) { return '_neutral_';}"
             "else if (v< .08) { return 'poslevel1';}"
             "else if (v< .12) { return 'poslevel2';}"
             "else             { return 'poslevel3';}")

#symbology for enrollment classes
unival_su = [
               {"value":"neglevel3", "label":"Less than -12%", "symbol":{"type":"esriSLS","color":[  0,  0,255,168],"width": 4}},
               {"value":"neglevel2", "label":"-12% to -8%"   , "symbol":{"type":"esriSLS","color":[  0,112,255,168],"width": 3}},
               {"value":"neglevel1", "label":"-8% to -4%"    , "symbol":{"type":"esriSLS","color":[115,178,255,168],"width": 2}},
               {"value":"_neutral_", "label":"-4% to 4%"     , "symbol":{"type":"esriSLS","color":[204,204,204,168],"width": 1}},
               {"value":"poslevel1", "label":"+4% to +8%"    , "symbol":{"type":"esriSLS","color":[255,190,190,168],"width": 2}},
               {"value":"poslevel2", "label":"+8% to +12%"   , "symbol":{"type":"esriSLS","color":[255,127,127,168],"width": 3}},
               {"value":"poslevel3", "label":"More than +12%", "symbol":{"type":"esriSLS","color":[230,  0,  0,168],"width": 4}}
            ]

#define sdf layer
sdfSegmentsWithCUData.spatial.plot(map_widget        = map3,
                                   renderer_type     = 'u-a',
                                   unique_values     = unival_su,
                                   arcade_expression = arcexp_su,
                                   default_symbol    = ""
                                  )

map3.layout.height='800px'
map3.legend=True

#add map title
display(Markdown('<h2><center>Segment Combo-Unit Changes (2019 vs 2015-2019)</center></h2>'))

#display the map
map3

<h2><center>Segment Combo-Unit Changes (2019 vs 2015-2019)</center></h2>

MapView(layout=Layout(height='800px', width='100%'), legend=True)

In [41]:
sdfSegmentsWithData

,OBJECTID,SEGID,BMP,EMP,LRS_ROUTE,ROUTE_ID,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,...,2019,2020,2021,2022,2023,2013-2017,2015-2019,2019-2023,All Years,COMPARE
0,1,0163_000.0,0.0,0.458,0163P,163,HWY 163,Arterial,5,Arterial,...,0.2458,0.2458,0.2459,0.0,0.2458,0.2472,0.2474,0.2458,0.2464,-0.0016
1,2,3225_000.0,0.0,1.68,3225P,3225,UTAH AVE,Arterial,6,Arterial,...,0.1494,0.1521,0.1561,0.0,0.1567,0.1524,0.152,0.1536,0.1527,-0.0026
2,3,3142_000.0,0.0,0.291,3142P,3142,RIVER RD,Arterial,4,Arterial,...,0.1282,0.1305,0.13,0.0,0.1289,0.1744,0.1517,0.1294,0.1517,-0.0235
3,4,3149_000.0,0.031,0.289,3149P,7,DESERT CANYONS PKWY,Arterial,2,Freeway,...,0.1282,0.1305,0.13,0.0,0.1289,0.1744,0.1517,0.1294,0.1517,-0.0235
4,5,0007_006.1,6.125,7.308,0007P,7,SOUTHERN PKWY,Arterial,2,Freeway,...,0.1556,0.0779,0.1169,0.0,0.1658,0.2695,0.1977,0.1339,0.1731,-0.0421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,8554,2080_009.3,9.272,9.636,2080P,2080,SEGO LILY DR,Arterial,5,Arterial,...,0.1143,0.1199,0.1057,0.0,0.1075,0.1246,0.1221,0.1119,0.1179,-0.0078
8478,8557,2161_003.2,3.207,3.772,2161P,2161,NEW BINGHAM HWY,Arterial,3,Arterial,...,0.1143,0.1199,0.1057,0.0,0.1075,0.1246,0.1221,0.1119,0.1179,-0.0078
8479,8560,WFRC_8220,0.0,0.0,0,0,AIRPORT RD,Arterial,4,Arterial,...,0.1143,0.1199,0.1057,0.0,0.1075,0.1246,0.1221,0.1119,0.1179,-0.0078
8480,8561,2161_003.4,3.4,3.772,2161P,2161,7800 S,Arterial,4,Arterial,...,0.1143,0.1199,0.1057,0.0,0.1075,0.1246,0.1221,0.1119,0.1179,-0.0078
